In [ ]:
# import modules and define functions
from pyscf import gto, scf
from matplotlib import pyplot as plt, ticker
import plotly.graph_objects as go
import matplotlib as mpl
import numpy as np

def make_rdm1(mo_coeff, mo_occ):
    mocc = mo_coeff[:,mo_occ>0]
    return np.dot(mocc*mo_occ[mo_occ>0], mocc.conj().T)

In [ ]:
# build H2 with minimal basis
ds = np.linspace(0.5, 2.0, 31)
ess = []
dss = []
c1ss = []
for d in ds:
    atom = 'H 0 0 0; H 0 0 %s' %(d)
    basis = 'sto-3g'
    charge = 0
    spin = 0
    H2 = gto.Mole()
    H2.build(atom=atom,basis=basis,charge=charge,spin=spin)
    # initiate RHF
    mf = scf.UHF(H2)
    es = []
    # run RHF SCF for 1001 c1 mo coefficients
    c1s = np.linspace(0,1,101)
    dm1_corr = ''
    mo_coeff_corr = ''
    mf.max_cycle = 0
    for c1 in c1s:
        c2 = abs((1 - c1 ** 2) ** 0.5)
        mo_coeff = np.array([[c1, c2], [c1, -c2]])
        mo_occ = np.array([2, 0])
        dm1 = make_rdm1(mo_coeff, mo_occ)
        e = mf.kernel(dm1)
        es.append(e)
    ess.append(es)
    dss.append(ds)
    c1ss.append(c1s)
        

In [ ]:
xlabel = 'c_1 coefficient squared'
ylabel = 'distance / A'
zlabel = 'Energy / Eh'
fig = go.Figure(
    data=[
        go.Surface(
            x=c1s ** 2 ,
            y=ds,
            z=ess
            )
            ],
    )
fig.update_layout(
    scene = dict(
        xaxis_title = xlabel,
        yaxis_title = ylabel,
        zaxis_title = zlabel,
        xaxis = dict(
            nticks=5, 
            range=[0,1.0],
            ),
        yaxis = dict(
            tickvals=np.linspace(0.5,2.0,4),
            range=[0.3,2.0]
            ),
        zaxis = dict(
            tickvals=list(range(-2,3)),
            range=[-2.2,2.0]
            ),
        aspectratio=dict(x=1, y=1, z=0.95)
    )
)
fig.show()